# Experiments of the paper: "Advancing SQL Injection Detection for High-Speed Data Centers: A Novel Approach Using Cascaded NLP"

## Initializations

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Check if the code is running on Google Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Clone and rename the repo
if IN_COLAB:
  print("You are using Google Colab.")
  !git clone https://github.com/gdrlab/cascaded-sqli-detection.git
  print("Renaming folder cascaded-sqli-detection to cascaded_sqli_detection")
  !mv cascaded-sqli-detection cascaded_sqli_detection
else:
  print("You are not using Google Colab. Make sure to install necessary libraries (use sqli-env.yml and Mamba) and Git (https://git-scm.com/downloads)")
  os.system('git clone https://github.com/gdrlab/cascaded-sqli-detection.git')
  print("Renaming folder cascaded-sqli-detection to cascaded_sqli_detection")
  os.rename('cascaded-sqli-detection', 'cascaded_sqli_detection')

%cd cascaded_sqli_detection

In [ ]:
from pathlib import Path
#main_folder = Path('/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230525_sqli_colab')
main_folder = Path().absolute()

In [ ]:
#%cd $main_folder
%ls -lah

/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230525_sqli_colab
total 304K
-rw------- 1 root root 3.6K May 23 14:01 classical_models.py
-rw------- 1 root root 1.2K May 30 11:02 config.ini
drwx------ 2 root root 4.0K May 11 11:42 datasets/
-rw------- 1 root root 7.7K May 23 14:01 ensemble_models.py
-rw------- 1 root root  20K May 25 12:48 experiments.py
-rw------- 1 root root  813 Apr 17 09:08 .gitignore
-rw------- 1 root root    3 Apr  3 11:59 LICENSE.md
-rw------- 1 root root 236K May 31 14:23 main.ipynb
drwx------ 2 root root 4.0K May 25 13:15 __pycache__/
-rw------- 1 root root 5.4K May 11 10:49 README.md
drwx------ 2 root root 4.0K May 25 13:06 results/
-rw------- 1 root root 1.3K May 25 15:31 run_classical_MLs.py
-rw------- 1 root root  318 May 11 10:49 sqli-env.yml
-rw------- 1 root root 6.3K May 25 14:56 templates.py
drwx------ 2 root root 4.0K Apr  3 11:59 trained_models/
drwx------ 2 root root 4.0K May 25 13:05 utils

Warning: BERT libraries had problems after Colab system is upgraded. The solution I found is to downgrade numpy. After running the following code, restart the session and check the numpy version. If the libraries are working just fine in future, you can remove the downgrade fix.

In [ ]:
#!pip install -q -U numpy==1.22 --ignore-installed

In [ ]:
import numpy as np
import sklearn
print("numpy: \t\t", np.__version__)
print("sklearn: \t", sklearn.__version__)
!python --version

numpy: 		 1.22.4
sklearn: 	 1.2.2
Python 3.10.11


In [ ]:
# A dependency of the preprocessing for BERT inputs
#!pip install  -q -U "tensorflow-text==2.8.*"
!pip install  -q -U "tensorflow-text"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 83.6 MB/s eta 0:00:00


In [ ]:
#!pip install -q tf-models-official==2.7.0
!pip install -q tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 23.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.8 MB/s eta 0:00:00


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.2.2.


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from pathlib import Path
from datetime import datetime
import os
import shutil


import matplotlib.pyplot as plt

import pandas as pd
import time


import time
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

from experiments import evaluate, save_results

print('Pandas v', pd.__version__)
#tf.get_logger().setLevel('ERROR')

Pandas v 1.5.3


In [ ]:
 !nvidia-smi

Wed May 31 14:24:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import configparser
from templates import DataManager, logger
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [ ]:
config.get('models', 'classic_models').split(',')

['MultinomialNB']

In [ ]:
data_manager = DataManager(config)

In [ ]:
#%%script echo skipping
results_dir = Path(config['results']['dir'])

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
file_name = f'results_{currentTime}.csv'
output_file = main_folder / results_dir / file_name
proposed_test_results_file = main_folder / results_dir / f'proposed_method_results_{currentTime}.csv'
print(f"All results:{output_file.absolute()}")
print(f"Proposed method results:{proposed_test_results_file.absolute()}")



All results:/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230525_sqli_colab/results/results_230531_142518.csv
Proposed method results:/content/drive/MyDrive/Akademik/Research and Projects/Sakir Hoca Projects/AI Security Intelligence/Codes/20230525_sqli_colab/results/proposed_method_results_230531_142518.csv


## Classical ML (Single NLP) tests

In [ ]:
!python run_classical_MLs.py -o $file_name

05-30-2023 11:04:54 l:27| Loaded dataset with 30609 rows.
Running the tests for seed: 13
Feature extractors:   0% 0/1 [00:00<?, ?it/s]05-30-2023 11:04:54 l:83| running 1 of 1 feature extractors.
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(

Models:   0% 0/1 [00:00<?, ?it/s]05-30-2023 11:04:56 l:91| running 1 of 1 models.
05-30-2023 11:04:56 l:151| Training model: MultinomialNB
05-30-2023 11:04:56 l:159| Ended training MultinomialNB in: 0.01803496399998039s
Models: 100% 1/1 [00:00<00:00, 29.6

## LLM (BERT) tests

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [ ]:
# Manual selection
bert_model_name_list = ['bert_en_uncased_L-12_H-768_A-12', 
                        'bert_en_cased_L-12_H-768_A-12',
                        'small_bert/bert_en_uncased_L-2_H-128_A-2',
                        'small_bert/bert_en_uncased_L-4_H-512_A-8',
                        'small_bert/bert_en_uncased_L-8_H-128_A-2',
                        'small_bert/bert_en_uncased_L-12_H-768_A-12',
                        'bert_multi_cased_L-12_H-768_A-12',
                       'albert_en_base',
                        'electra_base',
                        'electra_small']

In [ ]:
def build_classifier_model(local_tfhub_handle_preprocess, local_tfhub_handle_encoder):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Payload')
  preprocessing_layer = hub.KerasLayer(local_tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(local_tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
data_manager.x_train.shape[0]

24487

In [ ]:
#main_dir = Path('/content/drive/MyDrive/Colab Notebooks/kasim/2023 Feb Bert SQLi/')
# 'SQLiV3_train.tsv', 'SQLiV3_test.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_testing.tsv'
# 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'NEW_normal_plus_waf_bypass_dataset_training.tsv'
#train_file_name = 'SQLiV3_train.tsv'
#test_file_name = 'SQLiV3_test.tsv'
#train_file = Path(main_dir / train_file_name)
#test_file = Path(main_dir / test_file_name)

#print(f'Train file exists: {train_file.is_file()}')
#print(f'Test file exists: {test_file.is_file()}')


currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
#recording = Recorder(main_folder / results_dir / f'results_BERT_{currentTime}.csv')


AUTOTUNE = tf.data.AUTOTUNE
batch_size = 16
#seed = 42

x_train = data_manager.x_train
y_train = data_manager.y_train
x_test = data_manager.x_test
y_test = data_manager.y_test

# Create TensorFlow datasets for the training and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(8)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(8)

# Cache the training and validation datasets
train_dataset = train_dataset.cache()
val_dataset = val_dataset.cache()


# Prefetch the training and validation datasets
train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_ds = val_dataset.prefetch(buffer_size=AUTOTUNE)



#train_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = test_csv_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = val_ds

class_names = ['payload','label']



In [ ]:
tmp = train_ds.take(1)
for i in next(iter(tmp)):
  print(i)

tf.Tensor(
[b'9869'
 b"1'||  (  select 'mdqc' where 4533  =  4533 and 8189  =    (  select count  (  *  )   from sysibm.systables as t1,sysibm.systables as t2,sysibm.systables as t3  )  --"
 b"1'   )    )     )   and sleep  (  5  )  #"
 b'1 where 8578  =  8578 and 4770  =  4474--'
 b'SELECT COUNT ( officer ) FROM suddenly'
 b"-5597'  )   or make_set  (  2490  =  2164,2164  )  "
 b'SELECT TOP 50 PERCENT * FROM vote SELECT * FROM simplest FETCH FIRST 50 PERCENT ROWS ONLYSELECT TOP 3 * FROM test'
 b'calle alfred nobel, 42 7-g'], shape=(8,), dtype=string)
tf.Tensor([0 1 1 1 0 1 0 0], shape=(8,), dtype=int64)


In [ ]:
# Data manager _ split with seed
from sklearn.model_selection import train_test_split

def split_dataset(seed=42):
  split_ratio = float(data_manager.config['data_manager']['split_ratio'])

  data_manager.train, data_manager.test = train_test_split(
    data_manager.dataset, test_size=split_ratio, random_state=seed)

  data_manager.x_train = data_manager.train['payload'].values
  data_manager.x_test = data_manager.test['payload'].values
  data_manager.y_train = data_manager.train['label'].values
  data_manager.y_test = data_manager.test['label'].values

  data_manager.notes = {
    'seed': seed,
    'split_ratio': split_ratio,
    'train_size': len(data_manager.train),
    'test_size': len(data_manager.test),
  }

#split_dataset(seed=666)
#print(data_manager.notes['seed'])

In [ ]:
seeds = [13, 27, 42, 72, 84]# 34, 1984, 1994, 77]
#seeds = [13]
for seed in seeds:
  print('######################################')
  print(f"seed:{seed}")
  split_dataset(seed=seed)
  print(data_manager.notes['seed'])

  x_train = data_manager.x_train
  y_train = data_manager.y_train
  x_test = data_manager.x_test
  y_test = data_manager.y_test

  # Create TensorFlow datasets for the training and validation sets
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
  val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

  # Cache the training and validation datasets
  train_dataset = train_dataset.cache()
  val_dataset = val_dataset.cache()


  # Prefetch the training and validation datasets
  train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
  test_ds = val_dataset.prefetch(buffer_size=AUTOTUNE)

  class_names = ['payload','label']


  for bert_model_name in bert_model_name_list:
    #bert_model_name = bert_model_name_list[0]
    print('****************************************************')
    #recording.set_current_method(f"{bert_model_name}" )
    tfhub_handle_encoder = map_name_to_handle[bert_model_name]
    tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

    print(f'BERT model selected           : {tfhub_handle_encoder}')
    print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)
    classifier_model = build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder)

    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()

    epochs = 10 #modify this
    #steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    steps_per_epoch = data_manager.x_train.shape[0]
    num_train_steps = (steps_per_epoch * epochs) #//200 
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')

    classifier_model.compile(optimizer=optimizer,
                            loss=loss,
                            metrics=metrics)


    print(f'Training model with {tfhub_handle_encoder}')
    start_time = time.time()
    history = classifier_model.fit(x=train_ds, #train_ds.take(2)
                                  validation_data=test_ds, # validation_data=test_ds.take(2),
                                  epochs=epochs)
    stop_time = time.time()
    training_time = (stop_time - start_time)*1000 / data_manager.notes['train_size']

    loss, accuracy = classifier_model.evaluate(test_ds)

    print(f'Loss: {loss}')
    print(f'Accuracy: {accuracy}')

    start_time = time.time()
    y_pred = classifier_model.predict(test_ds)
    stop_time = time.time()
    testing_time = (stop_time - start_time)*1000 / data_manager.notes['test_size']
    y_pred_np = tf.cast(tf.sigmoid(y_pred) > 0.5, tf.int32).numpy()
    X, y_true = zip(*test_ds.unbatch())
    y_true_np = [y.numpy() for y in y_true]
    #precision, recall, f1, support = precision_recall_fscore_support(y_true_np, y_pred_np, average='binary')
    #tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()

    notes = {'feature_method': bert_model_name,'model': bert_model_name, 
            'seed': data_manager.notes['seed'], 
            'split_ratio': data_manager.notes['split_ratio'],
            'train_size': data_manager.notes['train_size'],
            'test_size': data_manager.notes['test_size'],
            'extraction_time':0, 'feature_size':0,
            'train_time': training_time,
            'pred_time': testing_time, 'dataset': config['dataset']['file']}
    result = evaluate(y_true_np, y_pred_np, notes)
    print(result)
    save_results([result], output_file)


######################################
seed:13
13
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/10
1531/1531 [==============================] - 151s 85ms/step - loss: 0.3231 - binary_accuracy: 0.8525 - val_loss: 0.0111 - val_binary_accuracy: 0.9977
Epoch 2/10
1531/1531 [==============================] - 129s 84ms/step - loss: 0.0152 - binary_accuracy: 0.9967 - val_loss: 0.0104 - val_binary_accuracy: 0.9980
Epoch 3/10
1531/1531 [==============================] - 129s 84ms/step - loss: 0.0093 - binary_accuracy: 0.9981 - val_loss: 0.0107 - val_binary_accuracy: 0.9984
Epoch 4/10
1531/1531 [==============================] - 129s 84ms/step - loss: 0.0056 - binary_accuracy: 0.9988 - val_loss: 0.0205 - val_binary_ac

## The Proposed Cascade NLP (two-stage) SQLi Detection Tests

### First Stage - Classical ML

In [ ]:
from templates import FeatureExtractor
from classical_models import Classical_Model
from pprint import pprint
from experiments import evaluate

xgboost_threshold = 0.05 #prediction
scale_pos_weight = 5000.0 #xgboost model parameter

pass_aggressive_threshold = -0.3 #prediction
class_weights = {1: 0.999, 0: 0.001} #PassiveAggressiveClassifier model parameter

feature_method = 'tf-idf_ngram'
#model_name = 'XGBoost'
#model_name = 'PassiveAggressiveClassifier'

In [ ]:

# Train and evaluate the first stage
def first_stage_passive_aggressive():
  model_name = 'PassiveAggressiveClassifier'
  feature_extractor = FeatureExtractor(feature_method)
  start_time = time.time()
  feature_extractor.extract_features(
      data_manager.x_train, data_manager.x_test)
  stop_time = time.time()
  extraction_time = ((stop_time - start_time)*1000 
                    / (len(data_manager.x_train) + len(data_manager.x_test)) )

  model = Classical_Model(model_name, class_weight=class_weights) #MODIFY IT if you change the model
  model.feature_method = feature_extractor.method
  start_time = time.time()
  model.fit(
      feature_extractor.features['train'], 
      data_manager.y_train) #scale_pos_weight=5.0
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / feature_extractor.features['train'].shape[0]


  start_time = time.time()
  first_stage_y_pred = model.predict(feature_extractor.features['test'],  pass_aggressive_threshold=pass_aggressive_threshold)
  stop_time = time.time()

  testing_time = (stop_time - start_time)*1000 / feature_extractor.features['test'].shape[0]

  notes = {'feature_method': feature_extractor.method,'model': model_name, 
            'seed': data_manager.notes['seed'], 
            'split_ratio': data_manager.notes['split_ratio'],
            'train_size': data_manager.notes['train_size'],
            'test_size': data_manager.notes['test_size'],
            'extraction_time':extraction_time, 'feature_size': feature_extractor.features['train'].shape[1],
            'train_time': training_time,
            'pred_time': testing_time, 'dataset': config['dataset']['file'],
          'class_weights':class_weights,
          'pass_aggressive_threshold': pass_aggressive_threshold
          }
  # Save results to csv file

  result = evaluate(data_manager.y_test, first_stage_y_pred, notes=notes)
  pprint(result)
  save_results([result], proposed_test_results_file)

  # Extract the positive predicitions for the second stage
  first_stage_positive_preds = data_manager.x_test[(first_stage_y_pred == 1)]
  first_stage_positive_preds_true_labels = data_manager.y_test[(first_stage_y_pred == 1)]
  fs_pos_len = len(first_stage_positive_preds)
  nof_test_samples = len(data_manager.x_test)
  fs_rat = fs_pos_len/nof_test_samples
  print(f"Positive predicitions in the first stage: {fs_pos_len} out of {nof_test_samples}. Ratio:{fs_rat}")

  return first_stage_positive_preds, first_stage_positive_preds_true_labels

In [ ]:
# Train and evaluate the first stage
def first_stage_xgboost():
  model_name = 'XGBoost'
  feature_extractor = FeatureExtractor(feature_method)
  start_time = time.time()
  feature_extractor.extract_features(
      data_manager.x_train, data_manager.x_test)
  stop_time = time.time()
  extraction_time = ((stop_time - start_time)*1000 
                    / (len(data_manager.x_train) + len(data_manager.x_test)) )

  model = Classical_Model(model_name, scale_pos_weight=scale_pos_weight)
  model.feature_method = feature_extractor.method
  start_time = time.time()
  model.fit(
      feature_extractor.features['train'], 
      data_manager.y_train) #scale_pos_weight=5.0
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / feature_extractor.features['train'].shape[0]


  start_time = time.time()
  first_stage_y_pred = model.predict(feature_extractor.features['test'], xgboost_threshold=xgboost_threshold)
  stop_time = time.time()

  testing_time = (stop_time - start_time)*1000 / feature_extractor.features['test'].shape[0]

  notes = {'feature_method': feature_extractor.method,'model': model_name, 
            'seed': data_manager.notes['seed'], 
            'split_ratio': data_manager.notes['split_ratio'],
            'train_size': data_manager.notes['train_size'],
            'test_size': data_manager.notes['test_size'],
            'extraction_time':extraction_time, 'feature_size': feature_extractor.features['train'].shape[1],
            'train_time': training_time,
            'pred_time': testing_time, 'dataset': config['dataset']['file'],
          'scale_pos_weight':scale_pos_weight,
          'xgboost_threshold': xgboost_threshold
          }
  # Save results to csv file

  result = evaluate(data_manager.y_test, first_stage_y_pred, notes=notes)
  pprint(result)
  save_results([result], proposed_test_results_file)

  # Extract the positive predicitions for the second stage
  first_stage_positive_preds = data_manager.x_test[(first_stage_y_pred == 1)]
  first_stage_positive_preds_true_labels = data_manager.y_test[(first_stage_y_pred == 1)]
  fs_pos_len = len(first_stage_positive_preds)
  nof_test_samples = len(data_manager.x_test)
  fs_rat = fs_pos_len/nof_test_samples
  print(f"Positive predicitions in the first stage: {fs_pos_len} out of {nof_test_samples}. Ratio:{fs_rat}")

  return first_stage_positive_preds, first_stage_positive_preds_true_labels

### Second Stage - BERT

In [ ]:
def second_stage(first_stage_positive_preds, first_stage_positive_preds_true_labels):
  # Prepare the training Dataset 
  currentDateAndTime = datetime.now()
  currentTime = currentDateAndTime.strftime("%y%m%d_%H%M%S")
  #recording = Recorder(main_folder / results_dir / f'results_BERT_{currentTime}.csv')


  AUTOTUNE = tf.data.AUTOTUNE
  batch_size = 8
  #seed = 42

  x_train = data_manager.x_train
  y_train = data_manager.y_train
  x_test = data_manager.x_test
  y_test = data_manager.y_test

  # Create TensorFlow datasets for the training and validation sets
  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
  val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

  # Cache the training and validation datasets
  train_dataset = train_dataset.cache()
  val_dataset = val_dataset.cache()


  # Prefetch the training and validation datasets
  train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
  test_ds = val_dataset.prefetch(buffer_size=AUTOTUNE)

  class_names = ['payload','label']

  # First, train the BERT model.

  print('****************************************************')
  #bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'
  #bert_model_name = 'small_bert/bert_en_uncased_L-8_H-128_A-2'#smallest Recall
  bert_model_name = 'electra_base'#smallest Recall

  tfhub_handle_encoder = map_name_to_handle[bert_model_name]
  tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

  print(f'BERT model selected           : {tfhub_handle_encoder}')
  print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


  bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
  bert_model = hub.KerasLayer(tfhub_handle_encoder)
  second_stage_classifier_model = build_classifier_model(tfhub_handle_preprocess, tfhub_handle_encoder)

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()

  epochs = 10 #DEBUG MODIFY ME
  #steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
  steps_per_epoch = data_manager.x_train.shape[0]
  num_train_steps = (steps_per_epoch * epochs) #//200 
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

  second_stage_classifier_model.compile(optimizer=optimizer,
                          loss=loss,
                          metrics=metrics)


  print(f'Training model with {tfhub_handle_encoder}')
  start_time = time.time()
  history = second_stage_classifier_model.fit(x=train_ds, # x=train_ds.take(2), 
                                validation_data=test_ds, # validation_data=test_ds.take(2), 
                                epochs=epochs)
  stop_time = time.time()
  training_time = (stop_time - start_time)*1000 / data_manager.notes['train_size']

  # Prepare first stage outputs for the second stage
  print(len(first_stage_positive_preds)) #np array
  print(len(first_stage_positive_preds_true_labels)) # np array

  # Create TensorFlow datasets for the prediction set
  pred_dataset = tf.data.Dataset.from_tensor_slices(
      (first_stage_positive_preds, first_stage_positive_preds_true_labels)).batch(batch_size)

  # Cache the pred dataset
  pred_dataset = pred_dataset.cache()

  # Prefetch the pred datasets
  first_stage_pos_preds = pred_dataset.prefetch(buffer_size=AUTOTUNE)

  # Predict only positive outputs of the first stage
  start_time = time.time()
  y_pred = second_stage_classifier_model.predict(first_stage_pos_preds)#.take(2))
  stop_time = time.time()
  testing_time = (stop_time - start_time)*1000 / len(first_stage_positive_preds) 
  y_pred_np = tf.cast(tf.sigmoid(y_pred) > 0.5, tf.int32).numpy()
  X, y_true = zip(*first_stage_pos_preds.unbatch())# take(2).
  y_true_np = [y.numpy() for y in y_true]
  #precision, recall, f1, support = precision_recall_fscore_support(y_true_np, y_pred_np, average='binary')
  #tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()

  notes = {'feature_method': bert_model_name,'model': bert_model_name, 
            'seed': data_manager.notes['seed'], 
            'split_ratio': data_manager.notes['split_ratio'],
            'train_size': data_manager.notes['train_size'],
            'test_size': data_manager.notes['test_size'],
            'extraction_time':0, 'feature_size':0,
            'train_time': training_time,
            'pred_time': testing_time, 'dataset': config['dataset']['file']}
  result = evaluate(y_true_np, y_pred_np, notes)
  pprint(result)
  save_results([result], proposed_test_results_file)


In [ ]:
# Run first and second stages with different seeds:
seeds = [13, 27, 42, 72, 84]# 34, 1984, 1994, 77]
#seeds = [13]
for seed in seeds:
  print('######################################')
  print(f"seed:{seed}")
  split_dataset(seed=seed)
  print(data_manager.notes['seed'])
  #first_stage_y_pred, first_stage_positive_preds_true_labels = first_stage_xgboost()
  first_stage_y_pred, first_stage_positive_preds_true_labels = first_stage_passive_aggressive()
  second_stage(first_stage_y_pred, first_stage_positive_preds_true_labels)

######################################
seed:13
13


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.9893825547206795,
 'class_weights': {0: 0.001, 1: 0.999},
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.07508826811165868,
 'f1_score': 0.9858480296102765,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 29157,
 'fn': 6,
 'fp': 59,
 'model': 'PassiveAggressiveClassifier',
 'pass_aggressive_threshold': -0.3,
 'precision': 0.9746018080068877,
 'pred_time': 0.0005613857918570767,
 'recall': 0.9973568281938326,
 'seed': 13,
 'split_ratio': 0.2,
 'test_size': 6122,
 'tn': 3793,
 'tp': 2264,
 'train_size': 24487,
 'train_time': 0.03603076557036821}
Positive predicitions in the first stage: 2323 out of 6122. Ratio:0.3794511597517151
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.025398191993112353,
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0,
 'f1_score': 0.0,
 'feature_method': 'bert_multi_cased_L-12_H-768_A-12',
 'feature_size': 0,
 'fn': 2264,
 'fp': 0,
 'model': 'bert_multi_cased_L-12_H-768_A-12',
 'precision': 0.0,
 'pred_time': 2.276095859562073,
 'recall': 0.0,
 'seed': 13,
 'split_ratio': 0.2,
 'test_size': 6122,
 'tn': 59,
 'tp': 0,
 'train_size': 24487,
 'train_time': 78.18969141085063}
######################################
seed:27
27


...

{'accuracy': 0.025795356835769563,
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0,
 'f1_score': 0.0,
 'feature_method': 'bert_multi_cased_L-12_H-768_A-12',
 'feature_size': 0,
 'fn': 2266,
 'fp': 0,
 'model': 'bert_multi_cased_L-12_H-768_A-12',
 'precision': 0.0,
 'pred_time': 2.2819560863596555,
 'recall': 0.0,
 'seed': 27,
 'split_ratio': 0.2,
 'test_size': 6122,
 'tn': 60,
 'tp': 0,
 'train_size': 24487,
 'train_time': 77.8370571880346}
######################################
seed:42
42


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.98921920940869,
 'class_weights': {0: 0.001, 1: 0.999},
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.07432337980922495,
 'f1_score': 0.9856521739130436,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 29216,
 'fn': 6,
 'fp': 60,
 'model': 'PassiveAggressiveClassifier',
 'pass_aggressive_threshold': -0.3,
 'precision': 0.9742157284056725,
 'pred_time': 0.0005724849906554996,
 'recall': 0.9973603167619886,
 'seed': 42,
 'split_ratio': 0.2,
 'test_size': 6122,
 'tn': 3789,
 'tp': 2267,
 'train_size': 24487,
 'train_time': 0.03616733997348242}
Positive predicitions in the first stage: 2327 out of 6122. Ratio:0.38010454099967333
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061 [

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


{'accuracy': 0.9877491016007841,
 'class_weights': {0: 0.001, 1: 0.999},
 'dataset': 'SQLiV3.tsv',
 'extraction_time': 0.07466779335792414,
 'f1_score': 0.9837837837837837,
 'feature_method': 'tf-idf_ngram',
 'feature_size': 29275,
 'fn': 6,
 'fp': 69,
 'model': 'PassiveAggressiveClassifier',
 'pass_aggressive_threshold': -0.3,
 'precision': 0.9705631399317406,
 'pred_time': 0.0005537526586483367,
 'recall': 0.9973695747479175,
 'seed': 72,
 'split_ratio': 0.2,
 'test_size': 6122,
 'tn': 3772,
 'tp': 2275,
 'train_size': 24487,
 'train_time': 0.03388086033027111}
Positive predicitions in the first stage: 2344 out of 6122. Ratio:0.38288141130349557
****************************************************
BERT model selected           : https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3
Training model with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3
Epoch 1/10
3061/3061